In [1]:
import pandas as pd
import time
import math
from datetime import datetime
import numpy as np
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
import multiprocessing as mp
from multiprocessing import Pool
import time
from collections import defaultdict
from collections import Counter

from lib.FriendBasedCF import FriendBasedCF
from lib.KernelDensityEstimation import KernelDensityEstimation
from lib.AdditiveMarkovChain import AdditiveMarkovChain
from lib.metrics import precisionk, recallk

In [2]:
data = pd.read_csv('/mnt/c/Users/sarah/Downloads/bahan_proposal/loc-gowalla_totalCheckins.txt/Gowalla_totalCheckins.txt', sep="\t", header=None)
data.columns = ["User_ID", "Checkin_Time", "Latitude", "Longitude", 'Location_ID']

In [37]:
data

,User_ID,Checkin_Time,Latitude,Longitude,Location_ID
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878
...,...,...,...,...,...
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724


In [3]:
# group check-in data by POI and count the number of unique visitors
poi_visitors = data.groupby('Location_ID')['User_ID'].nunique()

# filter POIs with fewer than 10 visitors
filtered_pois = poi_visitors[poi_visitors >= 10].index.tolist()

# filter check-in data for users with at least 15 check-in POIs
user_checkins = data.groupby('User_ID')['Location_ID'].nunique()
filtered_users = user_checkins[user_checkins >= 15].index.tolist()

# filter check-in data to only include users with at least 15 POIs and POIs with at least 10 visitors
filtered_data = data[(data['User_ID'].isin(filtered_users)) & (data['Location_ID'].isin(filtered_pois))]

In [4]:
while((filtered_data.groupby('User_ID')['Location_ID'].nunique().min()<15) | (filtered_data.groupby('Location_ID')['User_ID'].nunique().min()<10)):
    
    # group check-in data by POI and count the number of unique visitors
    poi_visitors = filtered_data.groupby('Location_ID')['User_ID'].nunique()

    # filter POIs with fewer than 10 visitors
    filtered_pois = poi_visitors[poi_visitors >= 10].index.tolist()

    # filter check-in data for users with at least 15 check-in POIs
    user_checkins = filtered_data.groupby('User_ID')['Location_ID'].nunique()
    filtered_users = user_checkins[user_checkins >= 15].index.tolist()
    filtered_data = filtered_data[(filtered_data['User_ID'].isin(filtered_users)) & (filtered_data['Location_ID'].isin(filtered_pois))]


In [5]:
filtered_data['User_ID'].value_counts().describe()

count    18737.000000
mean        68.221914
std         93.352043
min         15.000000
25%         25.000000
50%         40.000000
75%         71.000000
max       1624.000000
Name: User_ID, dtype: float64

In [38]:
filtered_data.isnull().sum()

User_ID         0
Location_ID     0
Checkin_Time    0
Latitude        0
Longitude       0
dtype: int64

In [7]:
all_uids = filtered_data['User_ID'].unique().tolist()
new_uids = [(i, uid) for i, uid in enumerate(all_uids)]
new_uids_df = pd.DataFrame(new_uids, columns=['new_uid', 'User_ID'])

In [8]:
all_lids = filtered_data['Location_ID'].unique().tolist()
new_lids = [(i, lid) for i, lid in enumerate(all_lids)]
new_lids_df = pd.DataFrame(new_lids, columns=['new_lid', 'Location_ID'])

In [9]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278274 entries, 0 to 6440486
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   User_ID       1278274 non-null  int64  
 1   Checkin_Time  1278274 non-null  object 
 2   Latitude      1278274 non-null  float64
 3   Longitude     1278274 non-null  float64
 4   Location_ID   1278274 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 58.5+ MB


In [10]:
filtered_data = filtered_data.merge(new_uids_df, on='User_ID', how='left')

In [11]:
filtered_data = filtered_data.merge(new_lids_df, on='Location_ID', how='left')

In [12]:
filtered_data.drop(columns = ['User_ID','Location_ID'], inplace=True)
filtered_data.reset_index(drop = True)

,Checkin_Time,Latitude,Longitude,new_uid,new_lid
0,2010-10-19T23:55:27Z,30.235909,-97.795140,0,0
1,2010-10-18T22:17:43Z,30.269103,-97.749395,0,1
2,2010-10-17T23:42:03Z,30.255731,-97.763386,0,2
3,2010-10-17T19:26:05Z,30.263418,-97.757597,0,3
4,2010-10-16T18:50:42Z,30.274292,-97.740523,0,4
...,...,...,...,...,...
1278269,2010-10-14T00:40:14Z,37.795595,-122.394499,18736,162
1278270,2010-10-13T19:36:48Z,37.802363,-122.405827,18736,2719
1278271,2010-10-13T02:34:37Z,37.807792,-122.418305,18736,5760
1278272,2010-10-13T01:02:48Z,37.808792,-122.410269,18736,22245


In [13]:
filtered_data = (filtered_data[['new_uid', 'new_lid', 'Checkin_Time', 'Latitude', 'Longitude']]).rename(columns={'new_uid': 'User_ID', 'new_lid': 'Location_ID'})


In [14]:
filtered_data

,User_ID,Location_ID,Checkin_Time,Latitude,Longitude
0,0,0,2010-10-19T23:55:27Z,30.235909,-97.795140
1,0,1,2010-10-18T22:17:43Z,30.269103,-97.749395
2,0,2,2010-10-17T23:42:03Z,30.255731,-97.763386
3,0,3,2010-10-17T19:26:05Z,30.263418,-97.757597
4,0,4,2010-10-16T18:50:42Z,30.274292,-97.740523
...,...,...,...,...,...
1278269,18736,162,2010-10-14T00:40:14Z,37.795595,-122.394499
1278270,18736,2719,2010-10-13T19:36:48Z,37.802363,-122.405827
1278271,18736,5760,2010-10-13T02:34:37Z,37.807792,-122.418305
1278272,18736,22245,2010-10-13T01:02:48Z,37.808792,-122.410269


In [15]:
ready_to_split = (
    filtered_data.groupby(['User_ID', 'Location_ID'])
    .agg(
        Checkin_Time=('Checkin_Time', 'first'),
        Freq=('Location_ID', 'size'),
        Latitude=('Latitude', 'first'),
        Longitude=('Longitude', 'first')
    )
    .sort_values(['User_ID','Checkin_Time'], ascending = [True, False])
    .reset_index()
)


In [16]:
ready_to_split

,User_ID,Location_ID,Checkin_Time,Freq,Latitude,Longitude
0,0,0,2010-10-19T23:55:27Z,1,30.235909,-97.795140
1,0,1,2010-10-18T22:17:43Z,28,30.269103,-97.749395
2,0,2,2010-10-17T23:42:03Z,2,30.255731,-97.763386
3,0,3,2010-10-17T19:26:05Z,1,30.263418,-97.757597
4,0,4,2010-10-16T18:50:42Z,1,30.274292,-97.740523
...,...,...,...,...,...,...
821926,18736,17011,2010-10-14T03:36:25Z,1,37.799627,-122.408395
821927,18736,162,2010-10-14T00:40:14Z,1,37.795595,-122.394499
821928,18736,2719,2010-10-13T19:36:48Z,1,37.802363,-122.405827
821929,18736,5760,2010-10-13T02:34:37Z,1,37.807792,-122.418305


In [17]:
# Group the dataframe by User_ID
grouped_data = ready_to_split.groupby('User_ID')

# Get the count of rows for each User_ID
user_counts = grouped_data.size().reset_index(name='count')

# Filter the dataframe to only include rows where the User_ID appears at least twice
ready_to_split = ready_to_split[ready_to_split['User_ID'].isin(user_counts[user_counts['count'] >= 2]['User_ID'])]


In [18]:
ready_to_split.groupby('Location_ID')['User_ID'].nunique().min()

10

In [19]:
# Group the dataframe by User_ID
grouped_data = ready_to_split.groupby('User_ID')

# Get the count of rows for each User_ID
user_counts = grouped_data.size().reset_index(name='count')

# Calculate the size of each fold for each User_ID
user_counts['fold_size'] = user_counts['count'] // 5

# Merge the user_counts dataframe with the original dataframe
merged_data = pd.merge(ready_to_split, user_counts, on='User_ID')

# Determine the test set by taking the first fold for each User_ID
test_set = merged_data[merged_data.groupby('User_ID').cumcount() < merged_data['fold_size']]

# Remove unnecessary columns
test_set = test_set.drop(columns=['count', 'fold_size', 'Latitude', 'Longitude','Checkin_Time'])

# Create the train set by removing the test set rows
train_set = merged_data.drop(test_set.index).drop(columns=['count', 'fold_size', 'Latitude', 'Longitude','Checkin_Time'])


In [20]:
test_set

,User_ID,Location_ID,Freq
0,0,0,1
1,0,1,28
2,0,2,2
3,0,3,1
4,0,4,1
...,...,...,...
821897,18735,7548,5
821898,18735,7522,1
821915,18736,641,1
821916,18736,2248,1


In [21]:
# # Calculate the half of the count for each User_ID
# user_counts['half_count'] = user_counts['count'].apply(lambda x: math.ceil(x / 2))

# # Merge the user_counts dataframe with the original dataframe
# merged_data = pd.merge(ready_to_split, user_counts, on='User_ID')

# # Filter the dataframe to only include rows where the row number is greater than or equal to the difference between the count and the half count for each User_ID group
# train_set = ((merged_data[merged_data.groupby('User_ID').cumcount() >= (merged_data['count'] - merged_data['half_count'])]).drop(columns = ['count','half_count'])).drop(columns = ['Checkin_Time','Latitude','Longitude'])


In [22]:
train_set

,User_ID,Location_ID,Freq
23,0,23,1
24,0,24,1
25,0,25,1
26,0,26,1
27,0,27,1
...,...,...,...
821926,18736,17011,1
821927,18736,162,1
821928,18736,2719,1
821929,18736,5760,1


In [23]:
print((len(train_set) + len(test_set)), len(ready_to_split)) 

821931 821931


In [24]:
def convert_to_unix_time(date_string):
    date = datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%SZ')
    return date.timestamp()

In [25]:
# apply conversion function to the 'date' column
filtered_data['Checkin_Time'] = filtered_data['Checkin_Time'].apply(convert_to_unix_time)

In [26]:
filtered_data

,User_ID,Location_ID,Checkin_Time,Latitude,Longitude
0,0,0,1.287507e+09,30.235909,-97.795140
1,0,1,1.287415e+09,30.269103,-97.749395
2,0,2,1.287334e+09,30.255731,-97.763386
3,0,3,1.287318e+09,30.263418,-97.757597
4,0,4,1.287230e+09,30.274292,-97.740523
...,...,...,...,...,...
1278269,18736,162,1.286992e+09,37.795595,-122.394499
1278270,18736,2719,1.286973e+09,37.802363,-122.405827
1278271,18736,5760,1.286912e+09,37.807792,-122.418305
1278272,18736,22245,1.286907e+09,37.808792,-122.410269


In [27]:
new_checkins = filtered_data[['User_ID','Location_ID','Checkin_Time']]
new_checkins

,User_ID,Location_ID,Checkin_Time
0,0,0,1.287507e+09
1,0,1,1.287415e+09
2,0,2,1.287334e+09
3,0,3,1.287318e+09
4,0,4,1.287230e+09
...,...,...,...
1278269,18736,162,1.286992e+09
1278270,18736,2719,1.286973e+09
1278271,18736,5760,1.286912e+09
1278272,18736,22245,1.286907e+09


In [28]:
poi_coos_df = (ready_to_split[['Location_ID','Latitude','Longitude']]).drop_duplicates('Location_ID')
poi_coos_df

,Location_ID,Latitude,Longitude
0,0,30.235909,-97.795140
1,1,30.269103,-97.749395
2,2,30.255731,-97.763386
3,3,30.263418,-97.757597
4,4,30.274292,-97.740523
...,...,...,...
655798,32505,57.694411,11.919261
660547,32506,59.272602,17.916985
661503,32507,57.783184,14.160486
661857,32508,57.760561,19.006881


In [29]:
new_data_dir = '/mnt/c/Users/sarah/Downloads/Skripsi/Dataset/Gowalla_1/New_Dataset_8020/'
train_set.to_csv(new_data_dir + 'Gowalla_train.txt', sep='\t', header = None, index=False)
test_set.to_csv(new_data_dir + 'Gowalla_test.txt', sep='\t', header = None, index=False)
new_checkins.to_csv(new_data_dir + 'Gowalla_checkins.txt', sep='\t', header = None, index=False)
with open(new_data_dir + 'Gowalla_data_size.txt', 'w') as f:
    output_str = f"{new_checkins['User_ID'].max()+1} {new_checkins['Location_ID'].max()+1}"
    f.write(output_str)
poi_coos_df.to_csv(new_data_dir + 'Gowalla_poi_coos.txt', sep='\t', header = None, index=False)